# Projet de Web Scraping sur eBay

Ce notebook a pour objectif de réaliser le scraping de données sur eBay pour différentes catégories de produits (Laptops, Monitors, Smart Watches, Graphics Cards).
Pour ce faire, nous utilisons des requêtes asynchrones avec `aiohttp` et `asyncio` ainsi que `BeautifulSoup` pour parser le contenu HTML.
Les données extraites seront ensuite sauvegardées dans des fichiers CSV pour une exploitation ultérieure.

**Bibliothèques utilisées :**
- `aiohttp`, `asyncio` : Pour effectuer des requêtes HTTP asynchrones.
- `BeautifulSoup` (via `bs4`) : Pour parser le HTML.
- `csv` : Pour enregistrer les résultats dans des fichiers CSV.
- `random` : Pour introduire des délais aléatoires et simuler un comportement humain.
- `fake_useragent` : Pour générer des User-Agent aléatoires.
- `datetime` : Pour gérer les dates d'extraction.
- `os` : Pour la gestion des fichiers et dossiers.


In [ ]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import csv
import random
from fake_useragent import UserAgent
from datetime import datetime
import os


## Définition des Headers

La fonction `get_headers()` permet de générer des en-têtes HTTP avec un User-Agent aléatoire.
Cela aide à éviter d'être bloqué par eBay en simulant un comportement de navigateur standard.


In [ ]:
# Initialisation de fake_useragent pour obtenir des User-Agent aléatoires
ua = UserAgent()

def get_headers():
    return {
        'User-Agent': ua.random,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Referer': 'https://www.ebay.com/',
        'DNT': '1'
    }


## Scraping des Détails d'un Produit

La fonction asynchrone `scrape_product_details` reçoit une session, l'URL du produit et la catégorie du produit.
Elle effectue les actions suivantes :
- Attente d'un délai aléatoire pour simuler le comportement humain.
- Envoi d'une requête HTTP GET avec des headers.
- Parsing du contenu HTML avec BeautifulSoup.
- Extraction du titre, du prix et des spécifications spécifiques selon la catégorie.
- Retourne un dictionnaire contenant les détails du produit.


In [ ]:
async def scrape_product_details(session, product_url, category):
    try:
        # Attendre un délai aléatoire entre 2 et 5 secondes
        await asyncio.sleep(random.uniform(2, 5))
        headers = get_headers()

        async with session.get(product_url, headers=headers) as response:
            response.raise_for_status()
            soup = BeautifulSoup(await response.text(), 'html.parser')

            # Extraction du titre du produit
            title = soup.find('h1', class_='x-item-title__mainTitle')
            title = title.text.strip() if title else 'N/A'

            # Extraction du prix
            price = soup.find('div', class_='x-price-primary')
            price = price.text.strip() if price else 'N/A'

            # Extraction des spécifications
            specs = {}
            for spec in soup.find_all('div', class_='ux-labels-values__labels'):
                key = spec.text.strip()
                # Recherche de la valeur associée à la clé
                value_tag = spec.find_next('div', class_='ux-labels-values__values')
                value = value_tag.text.strip() if value_tag else 'N/A'
                specs[key] = value

            # Dictionnaire de base pour les informations communes
            product_details = {
                'Title': title,
                'Price': price,
                'Collection Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            # Ajout des spécificités en fonction de la catégorie
            if category == "Laptops":
                product_details.update({
                    'RAM': specs.get('RAM Size', 'N/A'),
                    'CPU': specs.get('Processor', 'N/A'),
                    'Model': specs.get('Model', 'N/A'),
                    'Brand': specs.get('Brand', 'N/A'),
                    'GPU': specs.get('GPU', 'N/A'),
                    'Screen Size': specs.get('Screen Size', 'N/A'),
                    'Storage': specs.get('SSD Capacity', 'N/A'),
                })
            elif category == "Monitors":
                product_details.update({
                    'Screen Size': specs.get('Screen Size', 'N/A'),
                    'Maximum Resolution': specs.get('Resolution', 'N/A'),
                    'Aspect Ratio': specs.get('Aspect Ratio', 'N/A'),
                    'Refresh Rate': specs.get('Refresh Rate', 'N/A'),
                    'Response Time': specs.get('Response Time', 'N/A'),
                    'Brand': specs.get('Brand', 'N/A'),
                    'Model': specs.get('Model', 'N/A'),
                })
            elif category == "Smart Watches":
                product_details.update({
                    'Case Size': specs.get('Case Size', 'N/A'),
                    'Battery Capacity': specs.get('Battery Capacity', 'N/A'),
                    'Brand': specs.get('Brand', 'N/A'),
                    'Model': specs.get('Model', 'N/A'),
                    'Operating System': specs.get('Operating System', 'N/A'),
                    'Storage Capacity': specs.get('Storage Capacity', 'N/A')
                })
            elif category == "Graphics Cards":
                product_details.update({
                    'Brand': specs.get('Brand', 'N/A'),
                    'Memory Size': specs.get('Memory Size', 'N/A'),
                    'Memory Type': specs.get('Memory Type', 'N/A'),
                    'Chipset/GPU Model': specs.get('Chipset/GPU Model', 'N/A'),
                    'Connectors': specs.get('Connectors', 'N/A')
                })

            print(f"Successfully scraped {category}: {title[:50]}...")
            return product_details

    except Exception as e:
        print(f"Error scraping {product_url}: {str(e)}")
        return None


## Scraping d'une Page de Recherche

La fonction `scrape_search_page` effectue le scraping d'une page de résultats pour un terme de recherche donné :
- Elle construit l'URL avec les paramètres nécessaires.
- Elle envoie une requête GET asynchrone.
- Elle parse le HTML pour extraire les URLs des produits présents sur la page.


In [ ]:
async def scrape_search_page(session, query, page, semaphore, category):
    async with semaphore:
        try:
            base_url = "https://www.ebay.com/sch/i.html"
            params = {'_nkw': query, '_sacat': 0, '_from': 'R40', '_pgn': page}
            headers = get_headers()

            async with session.get(base_url, params=params, headers=headers) as response:
                response.raise_for_status()
                soup = BeautifulSoup(await response.text(), 'html.parser')

                # Extraction des URLs des produits
                items = soup.find_all('div', class_='s-item__wrapper')
                product_urls = [item.find('a', class_='s-item__link')['href']
                                for item in items if item.find('a', class_='s-item__link')]

                print(f"Scraped page {page} for {category} ({len(product_urls)} products)")
                return product_urls

        except Exception as e:
            print(f"Error scraping page {page} for {category}: {str(e)}")
            return []


## Orchestration du Scraping pour Plusieurs Catégories

La fonction `scrape_ebay_search` permet de :
- Lancer le scraping sur plusieurs pages pour chaque catégorie.
- Rassembler toutes les URLs des produits puis récupérer leurs détails.
- Retourner un dictionnaire regroupant les produits par catégorie.


In [ ]:
async def scrape_ebay_search(categories, max_pages=1):
    all_products = {}
    semaphore = asyncio.Semaphore(2)  # Limiter le nombre de requêtes simultanées

    async with aiohttp.ClientSession() as session:
        for category, query in categories.items():
            print(f"\n{'=' * 30}\nStarting {category} scraping\n{'=' * 30}")
            tasks = [scrape_search_page(session, query, page, semaphore, category)
                     for page in range(1, max_pages + 1)]

            search_results = await asyncio.gather(*tasks)
            product_urls = [url for sublist in search_results for url in sublist]

            product_tasks = [scrape_product_details(session, url, category) for url in product_urls]
            products = await asyncio.gather(*product_tasks)

            all_products[category] = [p for p in products if p]
            print(f"\n{'=' * 30}\nCompleted {category} ({len(all_products[category])} items)\n{'=' * 30}")

    return all_products


## Sauvegarde des Données en CSV

Deux fonctions sont définies ici :
- `get_next_scrape_number` : Pour déterminer le numéro de scrape suivant dans le dossier de sauvegarde.
- `save_to_csv` : Pour enregistrer les données extraites dans un fichier CSV.
Les fichiers CSV sont organisés par catégorie et contiennent les informations extraites.


In [ ]:
def get_next_scrape_number(save_directory, category):
    """Détermine le prochain numéro de scrape pour un dossier donné."""
    scrape_number = 1
    for filename in os.listdir(save_directory):
        if filename.startswith(f"{category}_") and filename.endswith(".csv"):
            try:
                # Extraction du numéro de scrape à partir du nom du fichier
                current_number = int(filename.split('_scrape')[-1].split('.')[0])
                if current_number >= scrape_number:
                    scrape_number = current_number + 1
            except ValueError:
                continue
    return scrape_number

def save_to_csv(data, category, save_directory, fieldnames):
    # Formatage du nom du dossier et du fichier
    category_folder = category.lower().replace(' ', '_')
    category_filename = category_folder
    today_date = datetime.now().strftime('%Y_%m_%d')

    # Création du dossier de sauvegarde s'il n'existe pas
    category_directory = os.path.join(save_directory, category_folder)
    os.makedirs(category_directory, exist_ok=True)

    # Détermination du prochain numéro de scrape
    scrape_number = get_next_scrape_number(category_directory, category_filename)

    # Génération du nom du fichier CSV
    filename = os.path.join(category_directory, f"{category_filename}_{today_date}_scrape{scrape_number}.csv")

    # Sauvegarde des données au format CSV
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

    print(f"Saved {len(data)} {category} items to {filename}")


## Fonction Principale et Exécution du Scraping

La fonction `main` orchestre l'ensemble du processus :
1. Définition des catégories et des requêtes associées.
2. Lancement du scraping asynchrone sur plusieurs pages pour chaque catégorie.
3. Sauvegarde des résultats dans des fichiers CSV organisés par catégorie.

**Note :**
Si vous exécutez ce code dans un notebook Jupyter et rencontrez une erreur relative à l'event loop, utilisez le module `nest_asyncio` pour permettre l'exécution de boucles asynchrones imbriquées.


In [ ]:
async def main():
    categories = {
        "Laptops": "laptop",
        "Monitors": "monitor",
        "Smart Watches": "smart watch",
        "Graphics Cards": "graphics card"
    }

    max_pages = 18  # Nombre de pages à scraper par catégorie
    save_directory = "data/raw/ebay"

    print("\nStarting eBay scraping...")
    all_products = await scrape_ebay_search(categories, max_pages)

    # Définition des colonnes attendues pour chaque catégorie
    category_fields = {
        "Laptops": ['Title', 'Price', 'RAM', 'CPU', 'Model', 'Brand', 'GPU', 'Screen Size', 'Storage', 'Collection Date'],
        "Monitors": ['Title', 'Price', 'Screen Size', 'Maximum Resolution', 'Aspect Ratio', 'Refresh Rate', 'Response Time', 'Brand', 'Model', 'Collection Date'],
        "Smart Watches": ['Title', 'Price', 'Case Size', 'Battery Capacity', 'Brand', 'Model', 'Operating System', 'Storage Capacity', 'Collection Date'],
        "Graphics Cards": ['Title', 'Price', 'Brand', 'Memory Size', 'Memory Type', 'Chipset/GPU Model', 'Connectors', 'Collection Date']
    }

    # Sauvegarde des données pour chaque catégorie
    for category, products in all_products.items():
        if products:
            save_to_csv(products, category, save_directory, category_fields[category])

# Exécution de la fonction principale.
# Dans un script Python, on utiliserait :
# if __name__ == "__main__":
#     asyncio.run(main())
#
# Pour Jupyter Notebook, en cas d'erreur liée à l'event loop, on peut utiliser nest_asyncio :
try:
    asyncio.run(main())
except RuntimeError as e:
    import nest_asyncio
    nest_asyncio.apply()
    await main()


## Conclusion

Ce notebook vous permet de :
- **Scraper** des données de produits sur eBay pour différentes catégories de produits.
- **Sauvegarder** ces données dans des fichiers CSV organisés par catégorie.
- **Analyser** les premiers résultats (vous pouvez par la suite utiliser des bibliothèques comme `pandas` pour approfondir l'analyse).

N'oubliez pas d'installer les dépendances nécessaires (par exemple via `!pip install aiohttp bs4 fake_useragent nest_asyncio`) avant d'exécuter ce notebook.

Bonne utilisation et bon scraping !
